In [ ]:
import os
import yaml
import json
import pandas as pd
import numpy as np
from sqlalchemy import create_engine


def GetTable():
    db_url = ("postgresql+psycopg2://postgres:"
                "@localhost:5432/bonsai_app"
            )
    engine = create_engine(db_url)
    cols = ['Moisture Use', 'Drought Tolerance','Shade Tolerance','Adapted to Coarse Textured Soils',
            'Adapted to Fine Textured Soils','Adapted to Medium Textured Soils','Leaf Retention',
            'Active Growth Period']
    df_all = pd.DataFrame()
    # get distinct/unique values for each col for diagnostic
    for col in cols:
        query = f'SELECT DISTINCT "{col}" FROM plant_traits;'
        df = pd.read_sql(query, engine)
        df.rename(columns={df.columns[0]: col}, inplace=True)
        df_all = pd.concat([df_all, df.reset_index(drop=True)], axis=1)
    return df_all

df = GetTable()

In [5]:
df

,Moisture Use,Drought Tolerance,Shade Tolerance,Adapted to Coarse Textured Soils,Adapted to Fine Textured Soils,Adapted to Medium Textured Soils,Leaf Retention,Active Growth Period
0,Low,Medium,None,No,No,No,No,Summer and Fall
1,None,None,Intermediate,Yes,Yes,Yes,Yes,Fall
2,High,Low,Tolerant,NaN,NaN,NaN,NaN,"Spring, Summer, Fall"
3,Medium,None,Intolerant,NaN,NaN,NaN,NaN,None
4,NaN,High,NaN,NaN,NaN,NaN,NaN,"Fall, Winter and Spring"
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spring and Fall
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Year Round
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spring
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spring and Summer


In [ ]:

# Example dataframe from your data
data = {
    "Moisture Use": ["Low", "None", "High", "Medium", np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
    "Drought Tolerance": ["Medium", "None", "Low", "None", "High", np.nan, np.nan, np.nan, np.nan, np.nan],
    "Shade Tolerance": ["None", "Intermediate", "Tolerant", "Intolerant", np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
    "Adapted to Coarse Textured Soils": ["No", "Yes", np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
    "Adapted to Fine Textured Soils": ["No", "Yes", np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
    "Adapted to Medium Textured Soils": ["No", "Yes", np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
    "Leaf Retention": ["No", "Yes", np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
    "Active Growth Period": ["Summer and Fall", "Fall", "Spring, Summer, Fall", "None", "Fall, Winter and Spring",
                              "Spring and Fall", "Summer", "Year Round", "Spring", "Spring and Summer"]
}

df = pd.DataFrame(data)

# Columns that represent soil types
soil_cols = [
    "Adapted to Coarse Textured Soils",
    "Adapted to Fine Textured Soils",
    "Adapted to Medium Textured Soils"
]

# Final result dictionary
result = {}

# Handle soil columns specially
soil_mapping = {
    "Adapted to Coarse Textured Soils": "coarse",
    "Adapted to Fine Textured Soils": "fine",
    "Adapted to Medium Textured Soils": "medium"
}

soil_dict = {}
for col in soil_cols:
    # Get first non-null unique value
    vals = df[col].dropna().unique().tolist()
    if vals:
        # If multiple unique values exist, pick the first or decide on rules
        soil_dict[soil_mapping[col]] = vals[0]
    else:
        soil_dict[soil_mapping[col]] = None

result["soil"] = soil_dict

# Handle all other columns normally
for col in df.columns:
    if col not in soil_cols:
        unique_vals = df[col].dropna().unique().tolist()
        result[col] = unique_vals

# Display
print(result)


{'soil': {'coarse': 'No', 'fine': 'No', 'medium': 'No'}, 'Moisture Use': ['Low', 'None', 'High', 'Medium'], 'Drought Tolerance': ['Medium', 'None', 'Low', 'High'], 'Shade Tolerance': ['None', 'Intermediate', 'Tolerant', 'Intolerant'], 'Leaf Retention': ['No', 'Yes'], 'Active Growth Period': ['Summer and Fall', 'Fall', 'Spring, Summer, Fall', 'None', 'Fall, Winter and Spring', 'Spring and Fall', 'Summer', 'Year Round', 'Spring', 'Spring and Summer']}
